# Purpose: Pig Cell Analysis

### Purpose: To segment and quantify features of all the pig data

Created by: Hawley Helmbrecht

Creation Date: 06/4/2021

Last Update: 

Notes:
    
    I did this process 3 times. First only using one of the channels. Second time, using a max intensity projection of all the channels since they appear to all be GFAP. And the third time with the max intensity projection and then with the binary fill holes after the remove small objects.
    
    I will do a qualitative check of segmentation accuracy to chose between these three for the VAMPIRE analysis and data visualization later. The current notebook shows the process for the 3rd option.

*Step 1: Import Necessary Packages*

In [ ]:
import numpy as np
import pandas as pd
from scipy import ndimage

import skimage.filters
from skimage import morphology
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage import io
from skimage import measure 

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import watermark
import os
from PIL import Image

*Step 2: User Inputs*

In [ ]:
#replace the example path from my computer with the path to the image on your computer

cell_folder = '/Users/hhelmbre/Desktop/Hawley_Pig_Data'

image_type = '.tif'

*Step 3: Defining a Folder Cleaner Function to only Return Tif Images*

In [ ]:
def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

*Step 4: Get All Images in the Folder*

In [ ]:
arr = os.listdir(cell_folder)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, image_type)

In [ ]:
file_list

*Step 5: Segmenting and Calculating Region Features on All Images*

In [ ]:
properties_list = ('area', 'bbox_area', 'centroid', 'convex_area', 
                   'eccentricity', 'equivalent_diameter', 'euler_number', 
                   'extent', 'filled_area', 'major_axis_length', 
                   'minor_axis_length', 'orientation', 'perimeter', 'solidity')


In [ ]:
j = 0
for names in file_list:
        
    try:
        cell_im = io.imread(str(cell_folder + '/' + names))
        print('name', names, 'shape', cell_im.shape)
        shape = cell_im.shape
        
        #Some of the images did not have 3 channels but still appear to include only the GFAP stain
        if len(shape) == 3:
            green_cell_im = np.max(cell_im, axis=2)
        else:
            green_cell_im = cell_im
    
        thresh_li = skimage.filters.threshold_li(green_cell_im)
        binary_li = green_cell_im > thresh_li
        new_binary_li = morphology.remove_small_objects(binary_li, min_size=500)
        new_binary_li = ndimage.binary_fill_holes(new_binary_li)
        label_image = label(new_binary_li)
        
        #Saving the Segmented Image
        cell_im_name = str(cell_folder + '/' + names)
        size = len(cell_im_name)
        cell_im_mod = cell_im_name[:size - 4]
        im_to_save = Image.fromarray(new_binary_li)
        im_to_save.save(str(cell_im_mod + '_' + 'segmented.png'))
        
        #Feel free to add them here as well. The computational time is pretty efficient
        props = measure.regionprops_table(label_image, properties=(properties_list))

        if j == 0:
            df = pd.DataFrame(props)
            df['filename'] = names
        else:
            df2 = pd.DataFrame(props)
            df2['filename'] = names
            df = df.append(df2)
            
    except FileNotFoundError:
        continue

    j = 1

*Step 6: Caculating the Circularity*

In [ ]:
df['circularity'] = 4*np.pi*df.area/df.perimeter**2

*Step 7: Calculating the Aspect Ratio*

In [ ]:
df['aspect_ratio'] = df.major_axis_length/df.minor_axis_length

*Step 8: Add in a column for the ID*

In [ ]:
df['sample_id'] = df.filename.str[:4]

*Step 9: Add in a column for the image number*

In [ ]:
df['image_id'] = df.filename.str[-6:]

In [ ]:
df['image_id'] = df.image_id.str[:2]

In [ ]:
df

*Step 10: Saving as a CSV file*

In [ ]:
df.to_csv('/Users/hhelmbre/Desktop/Hawley_Pig_Data/06_04_2021_allim_features_maxint_fill2.csv')

*Step 11: Print Dependencies and State*

In [ ]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,wget

%watermark -u -n -t -z